In [49]:
import torch
import math

In [50]:
import torch.nn as nn
import torch.optim as optim

In [51]:
import sys, time

def print_(*values, sep=' ', end='\n', file=sys.stdout, flush=True):
    file.write(sep.join(map(str, values)) + end)
    if flush:
        sys.stdout.flush()
    
def cls_line():
    print_('', end='\r')

print_('Loading model...', end='')
time.sleep(1)
cls_line()
print_('Loading model....')

Loading model...

Loading model....


In [52]:
vocab_size = 100
n_embed = 64
max_tokens = 10


In [53]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000, device = torch.device('cuda')):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [54]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, num_heads=8, device=torch.device('cuda')):
        super(Encoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self._dropout = dropout
        self._num_heads = num_heads
        self.device = device

        self.attention = nn.MultiheadAttention(hidden_size, num_heads, dropout=dropout, device=device)
        self.ffwd = nn.Sequential(
            nn.Linear(hidden_size, hidden_size, device=device),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size, device=device),
        )
        self.add_norm = nn.Sequential(
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU()
        )
        self.add_norm2 = nn.Sequential(
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU()
        )

    def forward(self, x):
        res, _ = self.attention(x, x, x) 
        x = self.add_norm1(res + x)
        res = self.ffwd(x)
        x = self.add_norm2(res + x)
        return x

In [55]:
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, num_heads=8, device=torch.device('cuda')):
        super(Decoder, self).__init__()

        self._hidden_size = hidden_size
        self._input_size = input_size
        self._output_size = output_size
        self._dropout = dropout
        self._num_heads = num_heads
        self._device = device

        self.masked_attention = nn.MultiheadAttention(
            hidden_size, num_heads, dropout=dropout, device=device)
        self.enc_dec_attention = nn.MultiheadAttention(
            hidden_size, num_heads, dropout=dropout, device=device)
        self.ffwd = nn.Sequential(
            nn.Linear(hidden_size, hidden_size, device=device),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size, device=device)
        )
        self.add_norm1 = nn.Sequential(
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU()
        )
        self.add_norm2 = nn.Sequential(
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU()
        )
        self.add_norm3 = nn.Sequential(
            nn.LayerNorm(hidden_size, device=device),
            nn.ReLU()
        )

    def forward(self, x, hidden_state):
        res, _ = self.masked_attention.forward(x, x, x, 
                                               attn_mask=torch.tril(
                                                   torch.eye(self._hidden_size, 
                                                             self._hidden_size, 
                                                             dtype=torch.bool,
                                                             device=self._device))) #! test mask
        x = self.add_norm1(res + x)
        res, _ = self.enc_dec_attention.forward(x, hidden_state, hidden_state) #! check if its right
        x = self.add_norm2(res + x)
        res = self.ffwd(x)
        x = self.add_norm3(res + x)
        return x

In [56]:
class EncoderBlock(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, num_heads=8, n_encoders=6, device=torch.device('cuda')):
        super(EncoderBlock, self).__init__()
        
        self._hidden_size = hidden_size
        self._input_size = input_size
        self._output_size = output_size
        self._dropout = dropout
        self._num_heads = num_heads
        self._n_encoders = n_encoders
        self._device = device

        self.embedding = nn.Embedding(input_size, hidden_size, device=device)
        self.pos_encoder = PositionalEncoding(hidden_size, device=device)

        self._encoders = [Encoder(input_size, hidden_size, output_size, dropout, num_heads, device) for _ in range(n_encoders)] 

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        
        for encoder in self._encoders:
            x = encoder(x)
            
        return x

In [57]:
class DecoderBlock(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, num_heads=8, n_decoders=6, device=torch.device('cuda')):
        super(DecoderBlock, self).__init__()

        self._hidden_size = hidden_size
        self._input_size = input_size
        self._output_size = output_size
        self._dropout = dropout
        self._num_heads = num_heads
        self._n_decoders = n_decoders
        self._device = device

        self.embedding = nn.Embedding(input_size, hidden_size, device=device)
        self.pos_encoder = PositionalEncoding(hidden_size, device=device)
        self.linear = nn.Linear(hidden_size, output_size, device=device)
        self.softmax = nn.Softmax(dim=1)

        self._decoders = [Decoder(
            input_size, hidden_size, output_size, dropout, num_heads, device=device) for _ in range(n_decoders)]

    def forward(self, x, hidden_state):
        x = self.embedding(x)
        x = self.pos_encoder(x)

        for decoder in self._decoders:
            x = decoder(x, hidden_state)
        
        x = self.linear(x)
        x = self.softmax(x)

        return x

In [58]:
from dataclasses import dataclass

class DataProcessor:
    __slots__ = ['_processor']
    def __init__(self, processor) -> None:
        self._processor = processor
    
    def __call__(self, *args, **kwargs):
        return self._processor(*args, **kwargs)
    
@dataclass(frozen=True, slots=True)
class Model:
    _preprocessor: DataProcessor
    _ai: nn.Module
    _postprocessor: DataProcessor

    def __call__(self, *args, **kwargs):
        return self._postprocessor(self._ai(self._preprocessor(*args, **kwargs)))
    

Test(_ai=DecoderBlock(
  (embedding): Embedding(100, 64)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (linear): Linear(in_features=64, out_features=100, bias=True)
  (softmax): Softmax(dim=1)
))

In [59]:
# Training loop using Adam optimizer, CrossEntropyLoss and EarlyStopping
class Trainer:

    @property
    def metrics(self):
        return self._metrics

    @metrics.setter
    def metrics(self, metrics):
        self._metrics = metrics

    @property
    def epochs(self):
        return self._epochs

    @epochs.setter
    def epochs(self, epochs):
        if epochs <= 0:
            raise ValueError('Number of epochs must be greater than zero')
        elif not isinstance(epochs, int):
            raise TypeError('Number of epochs must be an integer')

        self._epochs = epochs

    @property
    def batch_size(self):
        return self._batch_size

    @batch_size.setter
    def batch_size(self, batch_size):
        if batch_size <= 0:
            raise ValueError('Batch size must be greater than zero')
        elif not isinstance(batch_size, int):
            raise TypeError('Batch size must be an integer')

        self._batch_size = batch_size

    @property
    def model(self):
        return self._model
    
    def __init__(self, metrics, epochs, batch_size, model) -> None:
        self.metrics = metrics
        self.epochs = epochs
        self.batch_size = batch_size
        self._model = model

    def _train_one_epoch(self):
        pass

    def _validate(self):
        pass

    def fit(self, optimizer, loss_function, callbacks, **settings):
        pass

    def report(self):
        pass

    def graphics(self):
        pass